In [52]:
import polars as pl
import sys
import yaml
import os
import numpy as np
import pandas as pd

In [2]:
sys.path.append('../src/')
project_root = os.path.dirname(os.getcwd())

In [3]:
# Directorios para los archivos de parámetros y los datos
parameters_directory = os.path.join(project_root, 'src', 'parameters')
data_raw_directory = os.path.join(project_root, 'data', 'raw')
data_processed_directory = os.path.join(project_root, 'data', 'processed')
data_features_directory = os.path.join(project_root, 'data', 'features')

In [4]:
# Lista todos los archivos YAML en el directorio especificado
yaml_files = [f for f in os.listdir(parameters_directory) if f.endswith('.yml')]

# Diccionario para guardar los parámetros cargados
parameters = {}

# Carga cada archivo YAML
for yaml_file in yaml_files:
    with open(os.path.join(parameters_directory, yaml_file), 'r') as file:
        data = yaml.safe_load(file)
        key_name = f'parameters_{yaml_file.replace(".yml", "")}'
        parameters[key_name] = data

PROCESSING DATA

In [5]:
# # Nodos
# from functions.processing import (validate_tags_pl, 
#                              validate_dtypes_pl, 
#                              change_names_pl, 
#                              change_dtype_pl,
#                              delete_accents_pl,
#                              standardize_binary_values_pl,
#                              impute_missing_values_pl)

In [6]:
# tag_dict_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['tag_dict_path'])
# raw_data_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['raw_data_path'])
# 
# tag_dict = pl.read_excel(tag_dict_path)
# data_raw = pl.read_csv(raw_data_path)

In [7]:
# prueba1 = validate_tags_pl(data_raw, tag_dict)

In [8]:
# prueba2 = validate_dtypes_pl(prueba1, tag_dict)

In [9]:
# prueba3 = change_names_pl(prueba2, tag_dict)

In [10]:
# prueba4 = change_dtype_pl(prueba3, tag_dict)

In [11]:
# prueba5 = delete_accents_pl(prueba4)

In [12]:
# prueba6 = standardize_binary_values_pl(prueba5, parameters['parameters_processing'])

In [13]:
# prueba7 = impute_missing_values_pl(prueba6)

FEATURING DATA

In [14]:
# Nodos
from functions.featuring import (new_features_pl,
                                 add_target_variable_pl,
                                 one_hot_encoding_pl,
                                 random_forest_selection_pl,
                                 conditional_entropy_selection_pl,
                                 intersect_top_features_pl)

In [15]:
processed_data_path = os.path.join(data_processed_directory, parameters['parameters_catalog']['processed_data_path'])

data_processed = pl.read_csv(processed_data_path)

In [16]:
prueba8 = new_features_pl(data_processed, parameters['parameters_featuring'])

INFO:functions.featuring:Iniciando la generación de nuevas características...
INFO:functions.featuring:Nuevas características generadas!


In [17]:
target_directory = os.path.join(project_root, 'data', 'processed')
target_path = os.path.join(target_directory, parameters['parameters_catalog']['target_column_path'])
target = pl.read_csv(target_path)
prueba9 = add_target_variable_pl(prueba8, target, parameters['parameters_featuring'])

INFO:functions.featuring:Añadiendo la variable objetivo al DataFrame...
INFO:functions.featuring:Variable objetivo añadida!


In [18]:
prueba10 = one_hot_encoding_pl(prueba9, parameters['parameters_featuring'])

INFO:functions.featuring:Iniciando One Hot Encoding...
INFO:functions.featuring:Empieza el proceso de categorización acumulativa para el campo 'tipo_trabajo'...
INFO:functions.featuring:Categorización acumulativa completada para el campo 'tipo_trabajo'!
INFO:functions.featuring:Empieza el proceso de categorización acumulativa para el campo 'estado_civil'...
INFO:functions.featuring:Categorización acumulativa completada para el campo 'estado_civil'!
INFO:functions.featuring:Empieza el proceso de categorización acumulativa para el campo 'educacion'...
INFO:functions.featuring:Categorización acumulativa completada para el campo 'educacion'!
INFO:functions.featuring:Empieza el proceso de categorización acumulativa para el campo 'tipo_contacto'...
INFO:functions.featuring:Categorización acumulativa completada para el campo 'tipo_contacto'!
INFO:functions.featuring:Empieza el proceso de categorización acumulativa para el campo 'mes_contacto'...
INFO:functions.featuring:Categorización acumula

In [19]:
prueba11 = random_forest_selection_pl(prueba10, parameters['parameters_featuring'])

INFO:functions.featuring:Iniciando la selección de características con Random Forest...
INFO:functions.featuring:Selección de características con Random Forest completada!


In [20]:
prueba12 = conditional_entropy_selection_pl(prueba10, parameters['parameters_featuring'])

INFO:functions.featuring:Iniciando la selección de características con Entropía Condicional...
INFO:functions.featuring:Selección de características con Entropía Condicional completada!


In [35]:
target = parameters['parameters_featuring']['ce_selection']['target']
id = parameters['parameters_featuring']['ce_selection']['id']

In [40]:
target

'target'

In [43]:
df = prueba10
df = df.drop(id)

In [53]:
def add_random_variables_pd(df: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega dos variables aleatorias al DataFrame

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame al que se agregarán las variables aleatorias.

    Returns
    -------
    pd.DataFrame
        DataFrame con las variables aleatorias agregadas.
    """

    df["var_aleatoria_uniforme"] = np.random.rand(len(df))
    df["var_aleatoria_entera"] = np.random.randint(1, 5, size=len(df))

    return df

In [46]:
def entropy(p):
    """
    Calcula la entropía de un conjunto de probabilidades.

    Parameters
    ----------
    p : list
        Lista de probabilidades.

    Returns
    -------
    float: Valor de la entropía.
    """
    return -np.sum([pi * np.log2(pi) if pi > 0 else 0 for pi in p])

In [50]:
# Entropía variable objetivo
des = df.group_by(target).agg(pl.len().alias('count'))
pr = des['count'] / df.height  # Obtener el número total de filas con df.height()
Ho = entropy(pr.to_numpy())

In [55]:
# Convertir df a pandas para agregar las variables aleatorias
df_pandas = df.to_pandas()
df_pandas = add_random_variables_pd(df_pandas)
# Convertir de nuevo a DataFrame de Polars
df = pl.DataFrame(df_pandas)

In [58]:
# Cálculo de entropía condicional
feature_names, feature_importance = [], []

for columna in df.columns:
    if columna == target:
        continue
    H = 0
    feature_names.append(columna)

    # Cálculo de entropía condicional en el bucle for
    grouped = df.group_by(columna).agg(pl.col(target).count().alias("count"))
    # Aseguramos que cada columna sea accesible por su nombre para evitar confusiones futuras
    grouped = grouped.with_columns(pl.col(columna).alias('value'))
    for row in grouped.rows():
        # Usamos índices de acuerdo a cómo están ordenadas las columnas en el DataFrame agrupado
        value, group_count = row[0], row[1]  # Asumiendo que 'columna' y 'count' son las primeras dos columnas
        df_i = df.filter(pl.col(columna) == value)
        des = df_i.group_by(target).agg(pl.len().alias('count'))
        pr = des['count'] / group_count
        Hcond = entropy(pr.to_numpy())
        prob = group_count / df.height  # Usamos la propiedad height de Polars directamente
        H += Hcond * prob

    feature_importance.append(Ho - H)

# Crear un DataFrame con la importancia de las características
data_entropia = pl.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# Ordenar el DataFrame por importancia de manera descendente
df_entropia = data_entropia.sort(by='Importance', descending=True)

In [76]:
df_entropia.filter(pl.col("Feature") == "var_aleatoria_uniforme")

Feature,Importance
str,f64
"""var_aleatoria_uniforme""",0.506296


In [77]:
df_entropia

Feature,Importance
str,f64
"""buro_max_deuda""",0.506296
"""ratio_ingresos_deuda""",0.506296
"""ratio_deuda_creditos""",0.506296
"""var_aleatoria_uniforme""",0.506296
"""marg_cont""",0.429893
…,…
"""cdt""",0.000001
"""educacion_otro""",8.4468e-7
"""segmento_preferente""",7.8588e-8


In [61]:
# Obtener la importancia de las variables aleatorias
random_var_imp_0_1 = \
df_entropia.filter(pl.col("Feature") == "var_aleatoria_uniforme").select("Importance").to_numpy()[0][0]
random_var_imp_1_4 = \
df_entropia.filter(pl.col("Feature") == "var_aleatoria_entera").select("Importance").to_numpy()[0][0]

In [62]:
random_var_imp_0_1

0.5062956958567797

In [63]:
random_var_imp_1_4

0.00014076528375817077

In [60]:
#prueba13 = intersect_top_features_pl(prueba11, prueba12, parameters['parameters_featuring'])